In [2]:
import csv
import numpy as np
import pandas as pd
import future

In [3]:
import pickle
with open('X_train.txt', 'rb') as f:
    X_train = pickle.load(f)
with open('X_test.txt', 'rb') as f:
    X_test = pickle.load(f)
with open('y_train.txt', 'rb') as f:
    y_train = pickle.load(f)
with open('y_test.txt', 'rb') as f:
    y_test = pickle.load(f)

In [4]:
from keras.models import load_model
from keras_contrib.layers.crf import CRF, crf_loss, crf_viterbi_accuracy
model = load_model('ner-CNN-bi-lstm-model-FULL-0.99.hdf5', custom_objects={'CRF':CRF, 'crf_loss': crf_loss,'crf_viterbi_accuracy': crf_viterbi_accuracy})


Using TensorFlow backend.
C:\Users\Kasutaja\Anaconda3\envs\myEnv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [1]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from keras.models import load_model
import csv
import numpy as np
import pandas as pd
import future

import pickle
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn_crfsuite.metrics import flat_classification_report

from math import nan


from keras.models import Model, Input
from keras.layers import LSTM,MaxPooling1D,Flatten,Embedding, Dense, TimeDistributed, Conv1D,Dropout, Bidirectional
import keras as k
from keras_contrib.layers import CRF


from keras.callbacks import ModelCheckpoint

DS = pd.read_csv("DataAnnotatedExampleFullV2.csv")
DS['tag'].fillna('unk', inplace=True)
data = DS[['text_ID','word','tag']]
data.columns = ['sentence_idx','word','tag']


Using TensorFlow backend.


In [2]:
len(DS)

17026984

In [7]:


# tags = ["unk","value","object"]
tags = []
for tag in set(data["tag"].values):
    if tag is nan or isinstance(tag, float):
        tags.append('unk')
    else:
        tags.append(tag)
n_tags = len(tags)

from future.utils import iteritems

tag2idx = {t: i for i, t in enumerate(tags)}
idx2tag = {v: k for k, v in iteritems(tag2idx)}


In [8]:
tag2idx

{'object': 0, 'unk': 1, 'value': 2}

In [9]:
idx2tag

{0: 'object', 1: 'unk', 2: 'value'}

In [10]:
DS["tag"].value_counts()

unk       16482502
value       430906
object      113576
Name: tag, dtype: int64

In [11]:
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out
test_pred = model.predict(X_test, verbose=1) 
import numpy as np
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)

6162/6162 [==============================] - 153s 25ms/step


In [12]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))

F1-score: 3.6%


In [13]:
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_labels, y_true=test_labels,labels=tags)
print(report)

C:\Users\Kasutaja\Anaconda3\envs\myEnv\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

      object       0.00      0.00      0.00      8243
         unk       0.00      0.00      0.00      9748
       value       0.98      0.99      0.99   1072683

    accuracy                           0.98   1090674
   macro avg       0.33      0.33      0.33   1090674
weighted avg       0.97      0.98      0.97   1090674



In [ ]:
TP = {}
TN = {}
FP = {}
FN = {}
for tag in tag2idx.keys():
    TP[tag] = 0
    TN[tag] = 0    
    FP[tag] = 0    
    FN[tag] = 0    

def accumulate_score_by_tag(gt, pred):
    """
    For each tag keep stats
    """
    if gt == pred:
        TP[gt] += 1
    elif gt != 'O' and pred == 'O':
        FN[gt] +=1
    elif gt == 'O' and pred != 'O':
        FP[gt] += 1
    else:
        TN[gt] += 1
for i, sentence in enumerate(X_test):
    y_hat = np.argmax(test_pred[0], axis=-1)
    gt = np.argmax(y_test[0], axis=-1)
    for idx, (w,pred) in enumerate(zip(sentence,y_hat)):
        accumulate_score_by_tag(idx2tag[gt[idx]],tags[pred])
for tag in tag2idx.keys():
    print(f'tag:{tag}')    
    print('\t TN:{:10}\tFP:{:10}'.format(TN[tag],FP[tag]))
    print('\t FN:{:10}\tTP:{:10}'.format(FN[tag],TP[tag]))
#Reference: https://www.kaggle.com/nikkisharma536/ner-with-bilstm-and-crf/notebook

In [ ]:
y_trainLabel = pred2label(y_train)
train_labelsargmaxList = list(itertools.chain.from_iterable(y_trainLabel))


In [ ]:
print(train_labelsargmaxList.count("value"))
print(train_labelsargmaxList.count("object"))
print(train_labelsargmaxList.count("unk"))

In [ ]:
import itertools
T
test_labelsargmaxList = list(itertools.chain.from_iterable(test_labels))
print(test_labelsargmaxList.count("value"))
print(test_labelsargmaxList.count("object"))
print(test_labelsargmaxList.count("unk"))


In [ ]:
pred_labelsargmaxList = list(itertools.chain.from_iterable(pred_labels))
print(pred_labelsargmaxList.count("value"))
print(pred_labelsargmaxList.count("object"))
print(pred_labelsargmaxList.count("unk"))
